In [3]:
import re
import tweepy
import pandas as pd
from tweepy import OAuthHandler
from textblob import TextBlob

In [5]:
# keys and tokens from the Twitter Dev Console
consumer_key = 'UPnncVfZPSqjZo5c8YeJEUrUz'
consumer_secret = 'auB2RIDAhcXmZBxRXjR7YALD5lFCJHNabEjeywTN4Qju3PwmFG'
access_token = '987462390-Xyaq4zPncj6SRz15JR9bdYlJQY6MKQUEcE8fCtRm'
access_token_secret = 'SqwAKFbYA5337MqijRUnsUP6kQOnTOFlQ2zBMrRTlbUHc'

# attempt authentication
try:
    # create OAuthHandler object
    auth = OAuthHandler(consumer_key, consumer_secret)
    # set access token and secret
    auth.set_access_token(access_token, access_token_secret)
    # create tweepy API object to fetch tweets
    api = tweepy.API(auth)
except:
    print("Error: Authentication Failed")

In [6]:
def clean_tweet(tweet):
    '''
    Utility function to clean tweet text by removing links, special characters
    using simple regex statements.
    '''
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

In [7]:
def get_tweet_sentiment(tweet):
    '''
    Utility function to classify sentiment of passed tweet
    using textblob's sentiment method
    '''
    # create TextBlob object of passed tweet text
    analysis = TextBlob(clean_tweet(tweet))
    # set sentiment
    if analysis.sentiment.polarity > 0:
        return 'positive'
    elif analysis.sentiment.polarity == 0:
        return 'neutral'
    else:
        return 'negative'

In [8]:
def get_tweets(username):
    fetched_tweets = []
    tweets = []

    try:
        # Get 100 last tweets of the indicated username
        for status in tweepy.Cursor(api.user_timeline, screen_name=username, tweet_mode="extended").items(100):
            fetched_tweets.append(status)

        # parsing tweets one by one
        for tweet in fetched_tweets:
            # empty dictionary to store required params of a tweet
            parsed_tweet = {}

            # saving text of tweet
            parsed_tweet['text'] = tweet.full_text
            # saving sentiment of tweet
            parsed_tweet['sentiment'] = get_tweet_sentiment(tweet.full_text)

            # appending parsed tweet to tweets list
            if tweet.retweet_count > 0:
                # if tweet has retweets, ensure that it is appended only once
                if parsed_tweet not in tweets:
                    tweets.append(parsed_tweet)
            else:
                tweets.append(parsed_tweet)

        # return parsed tweets
        return tweets
    except tweepy.TweepError as e:
        print("Whoops! Something went wrong here. The error code is " + str(e) + username)
        print("That account does not exist! Try again")

In [11]:
tweets = None

while tweets is None:
    username = input("Please, enter your twitter username: ")
    tweets = get_tweets(username)

# picking positive tweets from tweets
ptweets = [tweet for tweet in tweets if tweet['sentiment'] == 'positive']
# percentage of positive tweets
print("\n\nPositive tweets percentage: {} %".format(100*len(ptweets)/len(tweets)))

# picking negative tweets from tweets
ntweets = [tweet for tweet in tweets if tweet['sentiment'] == 'negative']

# percentage of negative tweets
print("Negative tweets percentage: {} %".format(100*len(ntweets)/len(tweets)))

# picking neutral tweets from tweets
neutral_tweets = [tweet for tweet in tweets if tweet['sentiment'] == 'neutral']

# percentage of neutral tweets
print("Neutral tweets percentage: {} % ".format(100*len(neutral_tweets)/len(tweets)))

# printing positive tweets
print("\n\nPositive tweets:")
for tweet in ptweets:
    print(tweet['text'])


# printing negative tweets
print("\n\nNegative tweets:")
for tweet in ntweets:
    print(tweet['text'])

# printing neutral tweets
print("\n\nNeutral tweets:")
for tweet in neutral_tweets:
    print(tweet['text'])



Positive tweets percentage: 8.080808080808081 %
Negative tweets percentage: 2.0202020202020203 %
Neutral tweets percentage: 89.8989898989899 % 


Positive tweets:
@harkonnen21 Para lo mínimo indispensable. Si eso puede funcionar sin impuestos, tampoco
2º El PIB a coste de factores se divide en rentas salariales y en excedente bruto de explotación (EBE). Y no todo el EBE es renta gravable. En particular,hay dos partidas que no podemos considerar como renta gravable:el consumo de capital fijo y las rentas inmobiliarias imputadas
Ok https://t.co/qIlKVefRqS
@lavadora810 Usted ha dicho que reflejar el coste de las carreteras hará que suban los precios. Y cobrar por el combustible o por la compra de un camión también? El precio de cualquier bien ha de reflejar su coste real y completo.
@JRVB90 Por qué ligas eso con el liberalismo en general (y no con el caso específico de EEUU) si durante ese período terminó aboliéndose en Reino Unido?
@Ismaelseijo16 @tfg27620 Y? Irrelevante para lo que es

In [10]:
# save information in a dataset
user = api.get_user(username)
user_id = user.id_str

# assign genres by sentiment
genres_by_sentiment = {"positive": ["Comedy", "Adventure"], "negative": ["Drama", "Romance"], "neutral": ["Sci-Fi", "Action", "Animation", "Family", "Fantasy", "Musical", "Thriller", "War", "Western", "Crime", "Superhero"]}

status = 'neutral'
if(len(neutral_tweets) <= len(ptweets) and len(neutral_tweets) <= len(ntweets)):
    if(len(ptweets) >= len(ntweets)):
        status = 'positive'
    else:
        status = 'negative'

diccionario = {'user_id': [user_id], 'user_name': [username], 'status': [status], 'genres': [genres_by_sentiment[status]]}

df_user = pd.DataFrame(diccionario)

df_user.to_csv('userSentiment.csv', header=False, index=False)